# Module 11: Monitor and troubleshoot an Azure Cosmos DB SQL API solution

- [[Learning path]](https://docs.microsoft.com/en-us/learn/paths/monitor-troubleshoot-azure-cosmos-db-sql-api-solution/?ns-enrollment-type=Collection&ns-enrollment-id=1k8wcz8zooj2nx)
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/25-monitor.html): Use Azure Monitor to analyze an Azure Cosmos DB SQL API account
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/26-sdk-troubleshoot.html): Troubleshoot an application using the Azure Cosmos DB SQL API SDK
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/28-key-vault.html): Store Azure Cosmos DB SQL API account keys in Azure Key Vault

## Demo setup

In [ ]:
Connect-AzAccount
Set-AzContext -Subscription "b895a719-7034-411a-9944-ff196d1f450f"
$connectionString = (Get-AzCosmosDBAccountKey -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned -Type "ConnectionStrings")["Primary SQL Connection String"]
$primaryMasterKey = (Get-AzCosmosDBAccountKey -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned -Type "Keys")["PrimaryMasterKey"]
$documentEndpoint = (Get-AzCosmosDBAccount -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned).DocumentEndpoint

In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: Microsoft.Azure.Cosmos, 3.26.0-preview"
#r "nuget: Azure.Identity, 1.5.0"
#r "nuget:Microsoft.DotNet.Interactive.Kql,*-*"

#!share --from pwsh connectionString
#!share --from pwsh primaryMasterKey
#!share --from pwsh documentEndpoint

## Measure performance in Azure Cosmos DB SQL API

### Understand Azure Monitor

Azure Monitor is used to monitor the Azure resource availability, performance, and operations metrics.

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/measure-performance-azure-cosmos-db-sql-api/media/2-monitor-cosmos-db.png)

### Measure throughput

The Total Request Units metric can then be used to analyze those operations with the highest throughput.

View the Total Request Unit metrics:

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/measure-performance-azure-cosmos-db-sql-api/media/3-monitor-total-request-units.png)

Filter the Total Request Units further:

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/measure-performance-azure-cosmos-db-sql-api/media/3-monitor-total-request-units-filter.png)

### Observe rate-limiting events

There are three main reasons why we get a 429 exception:
- Request rate is large.
- The request did not complete due to a high rate of metadata requests.
- The request did not complete due to a transient service error.

Review the Insights-Request charts for 429s

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/measure-performance-azure-cosmos-db-sql-api/media/4-monitor-429-exception.png)

Review the Insights-Request charts for hot partitions 

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/measure-performance-azure-cosmos-db-sql-api/media/4-monitor-hot-partition.png)

### Query logs

Diagnostics settings are used to collect Azure Diagnostic Logs  produced by Azure resources. These logs provide detailed resource operational data.

Create Azure Cosmos DB diagnostics settings

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/measure-performance-azure-cosmos-db-sql-api/media/5-monitor-diagnostics-settings.png)

Query that returns the count and the total request charged of the different Azure Cosmos DB operation types in the last hour.

```kusto
AzureDiagnostics 
| where TimeGenerated >= ago(1h)
| where ResourceProvider=="MICROSOFT.DOCUMENTDB" and Category=="DataPlaneRequests" 
| summarize OperationCount = count(), TotalRequestCharged=sum(todouble(requestCharge_s)) by OperationName
| order by TotalRequestCharged desc


CDBDataPlaneRequests
| where TimeGenerated >= ago(1h)
| summarize OperationCount = count(), TotalRequestCharged=sum(todouble(RequestCharge)) by OperationName
| order by TotalRequestCharged desc 
```

Create a query that returns a timechart graph for all successful (status 200) and rate limited (status 429) request in the last hour. The requests will be aggregated every 10 minutes.

```kusto
AzureDiagnostics 
| where TimeGenerated >= ago(1h)
| where ResourceProvider=="MICROSOFT.DOCUMENTDB" and Category=="DataPlaneRequests" 
| summarize requestcount=count() by statusCode_s, bin(TimeGenerated, 10m)
| render timechart


CDBDataPlaneRequests 
| where TimeGenerated >= ago(2h)
| summarize requestcount=count() by StatusCode, bin(TimeGenerated, 10m)
| render timechart 
```

## Monitor responses and events in Azure Cosmos DB SQL API

### Review common response codes

| **Status Code** | **Name** | **Description** |
| :--- | :--- | :--- |
| **200** | OK | List, Get, Replace, Patch, Query -> The operation was successful. |
| **201** | Created | The operation was successful. |
| **204** | No Content |The delete operation was successful. |
| **304** | Not Modified |The document requested wasn't modified since the specified eTag value in the If-Match header. The service returns an empty response body. |
| **400** | Bad Request | The JSON body is invalid. Check for missing curly brackets or quotes. |
| **403** | Forbidden | The operation couldn't be completed because the storage limit of the partition has been reached. |
| **404** | Not Found | The document no longer exists, that is, the document was deleted. |
| **408** | Request timeout | The operation did not complete within the allotted amount of time. This code is returned when a stored procedure, trigger, or UDF (within a query) does not complete execution within the maximum execution time. |
| **409** | Conflict | The `id` provided for the new document has been taken by an existing document. |
| **413** | Entity Too Large | The document size in the request exceeded the allowable document size in a request. |
| **429** | Too many requests | The collection has exceeded the provisioned throughput limit. Retry the request after the server specified retry after duration. For more information, see request units. |
| **500** | Internal Server Error | The operation failed because of an unexpected service error. Contact support.  |
| **503** | Service Unavailable | The operation couldn't be completed because the service was unavailable. This situation could happen because of network connectivity or service availability issues. It's safe to retry the operation. If the issue persists, contact support.|

### Understand transient errors

We can identify and troubleshoot Azure Cosmos DB service unavailable exceptions when our request returns status code 503. 

- Required ports are blocked: Verify that the following ports are enabled for the SQL API.

| **Connection mode** | **Supported protocol** | **Supported SDKs** | **API/Service port** |
| :--- | :--- | :--- | :--- |
| Gateway | HTTPS | All SDKs | SQL (443) |
| Direct | TCP | .NET SDK, Java SDK | When using public/service endpoints: ports in the 10000 through 20000 range. When using private endpoints: ports in the 0 through 65535 range |

- Client-side transient connectivity issues

```csharp
TransportException: A client transport error occurred: The request timed out while waiting for a server response. 
(Time: xxx, activity ID: xxx, error code: ReceiveTimeout [0x0010], base error: HRESULT 0x80131500
```

- Service Outage: Check the [Azure status](https://status.azure.com/status) page to see if there's an ongoing issue.


### Review rate limiting errors

Requests return status code 429 for the exception request rate too large status code, indicating that your requests against Azure Cosmos DB are being rate-limited.

```kusto
AzureDiagnostics
| where TimeGenerated >= ago(24h)
| where Category == "DataPlaneRequests"
| summarize throttledOperations = dcountif(activityId_g, statusCode_s == 429), totalOperations = dcount(activityId_g), totalConsumedRUPerMinute = sum(todouble(requestCharge_s)) by databaseName_s, collectionName_s, OperationName, requestResourceType_s, bin(TimeGenerated, 1min)
| extend averageRUPerOperation = 1.0 * totalConsumedRUPerMinute / totalOperations 
| extend fractionOf429s = 1.0 * throttledOperations / totalOperations
| order by fractionOf429s desc
```

Rate-limiting due to transient service error: Retrying the request is the only recommended solution.

### Configure Alerts

Azure Cosmos DB uses the Azure Monitor Service to set up and send alerts.

![image](https://docs.microsoft.com/en-us/azure/cosmos-db/media/create-alerts/configure-alert-logic.png)

Here's an example of an alert that will trigger if the storage for a logical partition key exceeds 70% of the 20 GB limit (has more than 14 GB of storage)

![image](https://docs.microsoft.com/en-us/azure/cosmos-db/media/how-to-alert-on-logical-partition-key-storage-size/alert-signal-logic.png)

### Audit security

Activity logs, which are automatically available, contain all write operations (PUT, POST, DELETE) for your Cosmos DB resources except read operations (GET).

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/monitor-responses-events-azure-cosmos-db-sql-api/media/6-activity-log.png)

## Implementing backup and restore for Azure Cosmos DB SQL API

### Evaluate periodic backup

Azure Cosmos DB takes automatic backups of your data at regular periodic intervals.

Backup Storage Redundancy
- Geo-redundant
- Zone-redundant
- Locally redundant

Change the default backup interval and retention period
- Backup Interval
- Backup Retention
- Backup storage redundancy

![image](https://docs.microsoft.com/en-us/azure/cosmos-db/media/configure-periodic-backup-restore/configure-backup-options-existing-accounts.png)

To request to restore a backup
- Open a request ticket or call the Azure support team.

![image](https://docs.microsoft.com/en-us/azure/cosmos-db/media/configure-periodic-backup-restore/backup-support-request-portal.png)

Consider restoring a backup when you...
- Delete the entire Azure Cosmos DB account.
- Delete one or more Azure Cosmos DB databases.
- Delete one or more Azure Cosmos DB containers.
- Delete or modify the Azure Cosmos DB items within a container. This specific case is typically - referred to as data corruption.

Costs of Extra backups:
- Two backups included with the account for free.
- Extra backups will be charged on a region-based backup-storing pricing. 

Manage your own backups:
- Azure Data Factory
- Change feed

### Configure continuous backup and recovery

When using the continuous backups mode, backups are continuously taken in every region where the Azure Cosmos DB account exists.

![image](https://docs.microsoft.com/en-us/azure/cosmos-db/media/migrate-continuous-backup/enable-backup-migration.png)

Backup Storage Redundancy
- Locally redundant by default
- Zone-redundant when using Availability zones

Change backup options
- Only option is to enable Continuous Backups
- Once set on a new or existing account can not be changed

Continuous backup mode charges
- Backup storage space 
- A separate charge will be added every time a restore is started.

Limitations when using the continuous backup mode
- Azure Cosmos DB accounts using customer-managed keys are not supported.
- Multi-region write accounts not supported.
- You can't restore an account into a region where the source account did not exist.
- The retention period is 30 days and can't be changed.
- Can't modify or delete IAM policies when restore is in progress.
- Accounts that create unique indexes after the container is created are not supported.
- Point in time restore always restores to a new Azure Cosmos DB account.
- Collection's consistent indexes may still be rebuilding after completing the restore.
- Since TTL container properties are restored with the restore process, restores must be for timestamps before TTL properties were added to a container. This timestamp will prevent data from being deleted right after the restore.
- Azure Synapse Link and continuous backup mode can't coexist in the same database account.

### Perform a point-in-time recovery

Point-in-time recovery will allow you to choose any timestamp within the up to 30-days backup retention period and restore a combination of Azure DB containers, databases, or the accounts.

Scenarios:
- Restore deleted account
- Restore data of an account in a particular region
- Recover from an accidental write or delete operation within a container with a known restore timestamp
- Restore an account to a previous point in time before the accidental delete of the database
- Restore an account to a previous point in time before the accidental delete or modification of the container properties

![image](https://docs.microsoft.com/en-us/azure/cosmos-db/media/restore-account-continuous-backup/restore-live-account-portal.png)

## Implement security in Azure Cosmos DB SQL API

### Implement network-level access control

Azure Cosmos DB supports IP-based access controls for inbound firewall support.

Configure an IP firewall by using the Azure portal

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/implement-security-azure-cosmos-db-sql-api/media/2-firewall-setting.png)

### Review data encryption options

Azure Cosmos DB now uses encryption at rest for all its databases, backups, and media. When Azure Cosmos DB data is in transit, or over the network, that data is also encrypted.

Azure Cosmos DB at rest and in transit encryption implementation

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/implement-security-azure-cosmos-db-sql-api/media/3-encryption-keys.png)

### Use role-based access control (RBAC)

Azure role-based access control (RBAC) is provided in Azure Cosmos DB to do common management operations.

| **Built-in role** | **Description** |
| :--- | :--- |
| DocumentDB Account Contributor | Can manage Azure Cosmos DB accounts. |
| Cosmos DB Account Reader | Can read Azure Cosmos DB account data. |
| Cosmos Backup Operator | Can submit a restore request for Azure portal for a periodic backup enabled database or a container. Can modify the backup interval and retention on the Azure portal. Can't access any data or use Data Explorer. |
| CosmosRestoreOperator | Can do restore action for Azure Cosmos DB account with continuous backup mode. |
| Cosmos DB Operator | Can provision Azure Cosmos accounts, databases, and containers. Can't access any data or use Data Explorer. |

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/implement-security-azure-cosmos-db-sql-api/media/4-use-role-based-access-control.png)

Custom roles provide users a way to create Azure role definitions with a custom set of resource provider operations. 

![image](https://docs.microsoft.com/en-us/azure/cosmos-db/media/how-to-setup-rbac/concepts.svg)

### Access account resources using AAD

Access account resources using AAD allows you to authenticate your data requests with an Azure Active Directory (Azure AD) identity.

Permission model

- Microsoft.DocumentDB/databaseAccounts/sqlDatabases/containers/*
- Microsoft.DocumentDB/databaseAccounts/sqlDatabases/containers/items/create
- Microsoft.DocumentDB/databaseAccounts/sqlDatabases/containers/items/*
- Microsoft.DocumentDB/databaseAccounts/readMetadata
- Microsoft.DocumentDB/databaseAccounts/sqlDatabases/containers/executeStoredProcedure

To use Azure Cosmos DB RBAC with the SDK, you'll no longer pass the primary key. You'll pass an instance of a TokenCredential class. 

```csharp
TokenCredential servicePrincipal = new ClientSecretCredential(
    "<azure-ad-tenant-id>",
    "<client-application-id>",
    "<client-application-secret>");
CosmosClient client = new CosmosClient("<account-endpoint>", servicePrincipal);
```

In situations where you want to force clients to connect to Azure Cosmos DB through RBAC exclusively, you have the option to disable the account's primary/secondary keys. When doing so, any incoming request using either a primary/secondary key or a resource token will be actively rejected.

In [ ]:
az ad sp create-for-rbac --name "cosmosclientapp" --sdk-auth

This will generate a service principal

{
  "clientId": "xxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
  "clientSecret": "xxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "subscriptionId": "b895a719-7034-411a-9944-ff196d1f450f",
  "tenantId": "xxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
  "activeDirectoryEndpointUrl": "https://login.microsoftonline.com",
  "resourceManagerEndpointUrl": "https://management.azure.com/",
  "activeDirectoryGraphResourceId": "https://graph.windows.net/",
  "sqlManagementEndpointUrl": "https://management.core.windows.net:8443/",
  "galleryEndpointUrl": "https://gallery.azure.com/",
  "managementEndpointUrl": "https://management.core.windows.net/"
}

In [ ]:
az cosmosdb sql role definition list --account-name cosmos-dp-420-sql-provisioned --resource-group rg-dp-420

In [ ]:
az cosmosdb sql role assignment create `
    --account-name cosmos-dp-420-sql-provisioned `
    --resource-group rg-dp-420 `
    --scope "/" `
    --principal-id "b4a58102-a65b-42a9-ac32-93676fc8d3a9" `
    --role-definition-id "00000000-0000-0000-0000-000000000001"

In [ ]:
using Microsoft.Azure.Cosmos;
using Azure.Identity;
using Azure.Core;

TokenCredential servicePrincipal = new ClientSecretCredential(
    "72f988bf-86f1-41af-91ab-2d7cd011db47", // <azure-ad-tenant-id>
    "b4a58102-a65b-42a9-ac32-93676fc8d3a9", // <client-application-id>
    "U-DWDXOOu7BVym7BHTPqsKxPNUPQDsHRfd");  // <client-application-secret>
CosmosClient client = new CosmosClient(documentEndpoint, servicePrincipal);

### Understand Always Encrypted

Always encrypted encrypts sensitive data like credit card numbers or payroll information inside client-side applications.

Always encrypted uses encryption keys and decryption policies.

Always encrypted requires that you create **data encryption keys (DEK)** ahead of time. The DEKs are created at client-side using the Azure Comsos DB SDK. These DEKs are stored in the Azure Cosmos DB service. The DEKs are defined at the database level so they can be shared across multiple containers. Each DEK you create can be used to encrypt only one property, or can be used to encrypt many properties. You can have multiple DEKs per databases.

A DEK must be wrapped by a **customer-managed key (CMK)** before it stored in Azure Cosmos DB. Since CMKs control the wrapping and unwrapping of the DEKs, they control the access to the data that is encrypted with those DEKs. CMK storage is designed as an extensible/plug-in model, with a default implementation that expects them to be stored in Azure Key Vault. The relationship between these components is displayed in the following diagram.

![image](https://docs.microsoft.com/en-us/learn/wwl-data-ai/implement-security-azure-cosmos-db-sql-api/media/6-always-encrypted-cmk-dek.png)

Create a container with encryption policy:

In [ ]:
#r "nuget: Microsoft.Azure.Cosmos.Encryption, 1.0.0"
#r "nuget: Azure.Identity, 1.5.0"
#r "nuget: Azure.Security.KeyVault.Keys, 4.3.0"

In [ ]:
using Microsoft.Azure.Cosmos;
using Microsoft.Azure.Cosmos.Fluent;
using Microsoft.Azure.Cosmos.Encryption;

CosmosClient client = new (connectionString);
Database database = client.GetDatabase("cosmicworks");

var path1 = new ClientEncryptionIncludedPath 
{ 
  Path = "/property1", 
  ClientEncryptionKeyId = "my-key", 
  EncryptionType = EncryptionType.Deterministic.ToString(), 
  EncryptionAlgorithm = "AEAD_AES_256_CBC_HMAC_SHA256"
}; 

var path2 = new ClientEncryptionIncludedPath 
{ 
  Path = "/property2", 
  ClientEncryptionKeyId = "my-key", 
  EncryptionType = EncryptionType.Randomized.ToString(), 
  EncryptionAlgorithm = "AEAD_AES_256_CBC_HMAC_SHA256"
}; 

await database.DefineContainer("my-container", "/partition-key")
  .WithClientEncryptionPolicy() 
  .WithIncludedPath(path1) 
  .WithIncludedPath(path2) 
  .Attach() 
  .CreateAsync();


## Demo teardown

In [ ]:
await database.DeleteAsync();